# Evaluating Artificial Neural Network
based on: Udemy DEEP LEARNING A-Z™ Course
### Setup
To setup, we need to have these packages installed: <br>
1- Tensorflow<br>
2- Keras<br>
3- Theano | we will not work with this <br>

### A classification problem
based on the multi-dimensional data

In [2]:
import tensorflow as tf
# Theano is a mathematical library based on numpy, that can run on the GPU
#import theano
import keras

# we will need these for the ANN
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# will need this for data pre-processing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# will need this for splitting the data set
from sklearn.model_selection import train_test_split

# will need it for the scaling!
from sklearn.preprocessing import StandardScaler

# will need to calculate the loss
from sklearn.metrics import confusion_matrix

################################################################################
# will need it for the cross validation
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


## Data pre-processing
Reading data from the CSV file and make it ready for the next steps.

### Imporet data

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
# we only need features from 3 to 12 (upper bound is not inclusive)
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values
print (X[0])

[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]


### Encoding categorical data
There are two variables that are categorial, country and gender <br>
So, we convert them into one integer values! So if we apply it to country, it will replcae France, Germany, and Spain with 0,1,2! Nice!

In [4]:
# this will handle the country
labelEncoderX01 = LabelEncoder()
X[:,1] = labelEncoderX01.fit_transform(X[:,1])

# this will handle the gender
labelEncoderX02 = LabelEncoder()
X[:,2] = labelEncoderX02.fit_transform(X[:,2])

### Dummy coding
Now, we need to make some dummy coding! This will prevent the data to make this impression that categoprial values are actually impying any order. Spain is not higher or lower than France nor Germany.
We don't ned to do it with gender, since it only has two variables.
It basically make that 1,2,3 for countries into 3 different variables:
France  0, 0, 1
Germany 0, 1, 0
Spain   1, 0, 0
But we need to remove one of them, because we don't need the third one (if it is not German or French, then it is Spanish!)

In [5]:
oneHotEncoder = OneHotEncoder(categorical_features=[1])
X= oneHotEncoder.fit_transform(X).toarray()
X = X[:, 1:]

### Splitting the data into training and test
Easy peasy!

In [6]:
XTrain, XTest, yTrain, yTest = train_test_split(X,y, test_size = 0.2, random_state = 0)

### Feature Scaling
We must normalize the data, otherwise one feature is going to dominate the others! 

In [7]:
sc = StandardScaler()
# we first need to find the scale
XTrain = sc.fit_transform(XTrain)
# then we apply the same scale to the test set!
XTest = sc.transform(XTest)

# Cross Validation / K-Fold
This is a function that makes the ANN architecture like the previous section, but based on the Scikit_learn

In [70]:
def buildClassifier():
    classifier = Sequential()
    #classifier.add(Dense(units= 6, kernel_initializer= "uniform",activation = "relu",input_dim= 11))
    #classifier.add(Dense(units= 6, kernel_initializer= "uniform",activation = "relu"))
    #classifier.add(Dense(units= 1, kernel_initializer= "uniform",activation = "sigmoid"))
    #classifier.compile(optimizer = 'adam', loss ='binary_crossentropy',metrics = ['accuracy'])
    #classifier.compile(optimizer = 'rmsprop', loss ='binary_crossentropy',metrics = ['accuracy'])
    # fine-tuned!
    classifier.add(Dense(units= 6, kernel_initializer= "glorot_uniform",activation = "relu",input_dim= 11))
    classifier.add(Dense(units= 6, kernel_initializer= "glorot_uniform",activation = "relu"))
    classifier.add(Dense(units= 1, kernel_initializer= "glorot_uniform",activation = "sigmoid"))
    classifier.compile(optimizer = 'nadam', loss ='binary_crossentropy',metrics = ['accuracy'])

    return classifier

Do not add n_jobs = -1 if you use GPU, it will use only one CUDA core!

In [71]:
%%time
classifier =  KerasClassifier(build_fn = buildClassifier, batch_size= 500, epochs = 200, verbose=1)
# returns the 10 accuracies from the 10 fold cross validation
acuracies = cross_val_score(estimator = classifier,X = XTrain, y = yTrain, cv = 10)#, n_jobs = -1) 

Epoch 1/200
7200/7200 [==============================] - 2s 291us/step - loss: 0.8190 - acc: 0.3907
Epoch 2/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.7313 - acc: 0.5072
Epoch 3/200
7200/7200 [==============================] - 0s 18us/step - loss: 0.6727 - acc: 0.6208
Epoch 4/200
7200/7200 [==============================] - 0s 19us/step - loss: 0.6315 - acc: 0.7269
Epoch 5/200
7200/7200 [==============================] - 0s 19us/step - loss: 0.5980 - acc: 0.7822
Epoch 6/200
7200/7200 [==============================] - 0s 20us/step - loss: 0.5696 - acc: 0.7954
Epoch 7/200
7200/7200 [==============================] - 0s 19us/step - loss: 0.5459 - acc: 0.7971: 0s - loss: 0.5456 - acc: 0.800
Epoch 8/200
7200/7200 [==============================] - 0s 18us/step - loss: 0.5267 - acc: 0.7971
Epoch 9/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.5120 - acc: 0.7971
Epoch 10/200
7200/7200 [==============================] - 0s 17us/step - los

7200/7200 [==============================] - 0s 17us/step - loss: 0.3536 - acc: 0.8528
Epoch 83/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3532 - acc: 0.8522
Epoch 84/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3530 - acc: 0.8535
Epoch 85/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3527 - acc: 0.8544
Epoch 86/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3523 - acc: 0.8537
Epoch 87/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3521 - acc: 0.8549
Epoch 88/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3518 - acc: 0.8540
Epoch 89/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3515 - acc: 0.8549
Epoch 90/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3514 - acc: 0.8560
Epoch 91/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3510 - acc: 0.8547
Epoch 92/200


7200/7200 [==============================] - 0s 16us/step - loss: 0.3378 - acc: 0.8615
Epoch 164/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3378 - acc: 0.8622
Epoch 165/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3378 - acc: 0.8628
Epoch 166/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3375 - acc: 0.8621
Epoch 167/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3374 - acc: 0.8619
Epoch 168/200
7200/7200 [==============================] - 0s 15us/step - loss: 0.3373 - acc: 0.8617
Epoch 169/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3375 - acc: 0.8626
Epoch 170/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3372 - acc: 0.8632
Epoch 171/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3372 - acc: 0.8631
Epoch 172/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3370 - acc: 0.8633
Epoc

7200/7200 [==============================] - 0s 16us/step - loss: 0.4090 - acc: 0.8219
Epoch 45/200
7200/7200 [==============================] - 0s 15us/step - loss: 0.4083 - acc: 0.8232
Epoch 46/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.4077 - acc: 0.8232
Epoch 47/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.4071 - acc: 0.8250
Epoch 48/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.4065 - acc: 0.8247
Epoch 49/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.4058 - acc: 0.8250
Epoch 50/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.4052 - acc: 0.8258
Epoch 51/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.4046 - acc: 0.8263
Epoch 52/200
7200/7200 [==============================] - 0s 15us/step - loss: 0.4038 - acc: 0.8267
Epoch 53/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.4032 - acc: 0.8272
Epoch 54/200


7200/7200 [==============================] - 0s 17us/step - loss: 0.3323 - acc: 0.8657
Epoch 126/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3324 - acc: 0.8656
Epoch 127/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3320 - acc: 0.8664
Epoch 128/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3324 - acc: 0.8660
Epoch 129/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3320 - acc: 0.8663
Epoch 130/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3320 - acc: 0.8665
Epoch 131/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3319 - acc: 0.8662
Epoch 132/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3320 - acc: 0.8658
Epoch 133/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3319 - acc: 0.8665
Epoch 134/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3317 - acc: 0.8658
Epoc

...

Epoch 21/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.4118 - acc: 0.8314
Epoch 22/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.4096 - acc: 0.8314
Epoch 23/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.4076 - acc: 0.8319
Epoch 24/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.4057 - acc: 0.8332
Epoch 25/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.4041 - acc: 0.8335
Epoch 26/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.4023 - acc: 0.8333
Epoch 27/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.4006 - acc: 0.8336
Epoch 28/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3992 - acc: 0.8343
Epoch 29/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3978 - acc: 0.8340
Epoch 30/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3965 - acc: 0.8351


7200/7200 [==============================] - 0s 17us/step - loss: 0.3295 - acc: 0.8640
Epoch 103/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3294 - acc: 0.8636
Epoch 104/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3291 - acc: 0.8637
Epoch 105/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3291 - acc: 0.8639
Epoch 106/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3290 - acc: 0.8629
Epoch 107/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3288 - acc: 0.8633
Epoch 108/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3286 - acc: 0.8643
Epoch 109/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3286 - acc: 0.8649
Epoch 110/200
7200/7200 [==============================] - 0s 18us/step - loss: 0.3285 - acc: 0.8636
Epoch 111/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3285 - acc: 0.8646
Epoc

7200/7200 [==============================] - 0s 17us/step - loss: 0.3246 - acc: 0.8668
Epoch 184/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3245 - acc: 0.8674
Epoch 185/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3245 - acc: 0.8671
Epoch 186/200
7200/7200 [==============================] - 0s 15us/step - loss: 0.3244 - acc: 0.8671
Epoch 187/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3246 - acc: 0.8665
Epoch 188/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3244 - acc: 0.8669
Epoch 189/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3245 - acc: 0.8660
Epoch 190/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3245 - acc: 0.8669
Epoch 191/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3244 - acc: 0.8675
Epoch 192/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3243 - acc: 0.8667
Epoc

7200/7200 [==============================] - 0s 15us/step - loss: 0.3434 - acc: 0.8569
Epoch 65/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3431 - acc: 0.8587
Epoch 66/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3427 - acc: 0.8579
Epoch 67/200
7200/7200 [==============================] - 0s 18us/step - loss: 0.3423 - acc: 0.8579
Epoch 68/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3420 - acc: 0.8585
Epoch 69/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3417 - acc: 0.8579
Epoch 70/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3413 - acc: 0.8582
Epoch 71/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3411 - acc: 0.8581
Epoch 72/200
7200/7200 [==============================] - 0s 18us/step - loss: 0.3409 - acc: 0.8575
Epoch 73/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3407 - acc: 0.8567
Epoch 74/200


7200/7200 [==============================] - 0s 17us/step - loss: 0.3363 - acc: 0.8592
Epoch 146/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3361 - acc: 0.8586
Epoch 147/200
7200/7200 [==============================] - 0s 18us/step - loss: 0.3360 - acc: 0.8586
Epoch 148/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3361 - acc: 0.8597
Epoch 149/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3360 - acc: 0.8596
Epoch 150/200
7200/7200 [==============================] - 0s 16us/step - loss: 0.3360 - acc: 0.8593
Epoch 151/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3360 - acc: 0.8585
Epoch 152/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3358 - acc: 0.8594
Epoch 153/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3360 - acc: 0.8592
Epoch 154/200
7200/7200 [==============================] - 0s 17us/step - loss: 0.3360 - acc: 0.8594
Epoc

In [72]:
print(acuracies.mean())
print(acuracies.std())

0.858500003815
0.0105445518824


In [ ]:
500/500
0.859249990433
0.00624999776483
